In [ ]:
import dataset
from dataset import FusedSentenceMichigan
USE_REAL_DATA = False
DATASET =FusedSentenceMichigan(feature_options={},dataset_size=500, overlaps=[1000],use_real_data=USE_REAL_DATA)


In [ ]:
from dataset import load_segmentation_model
a,b = load_segmentation_model()

In [ ]:
# import importlib
# importlib.reload(dataset)
# from dataset import single_segmentation
# import torchaudio
# import torchaudio

# # Specify the file path
# file_path = "./resources/audio.wav"

# # Load the audio file
# waveform, sample_rate = torchaudio.load(file_path)
# waveform =  torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
# waveform = waveform[0].unsqueeze(0)
# text = ["你 hao jing tian mei you chi fan"]

# print(sample_rate)
# Audio(waveform.numpy(), rate=sample_rate)
# print(waveform.shape)
# single_segmentation(waveform,originallen=8)

In [ ]:
import importlib
importlib.reload(dataset)
from dataset import run_segmentation

SEGMENTATION_RESULTS, DATASET_PROPERTIES = run_segmentation(dataset=DATASET)

In [ ]:
DATASET_PROPERTIES

In [ ]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import processes_segementation_results_global
PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET = processes_segementation_results_global(SEGMENTATION_RESULTS,real=USE_REAL_DATA)

In [ ]:
PROCESSED_SEGEMENTATION_DATASET["diff"].value_counts(normalize=True)
PROCESSED_SEGEMENTATION_DATASET

In [ ]:
import pandas as pd
from dataset import convert_segmentations_to_index





PSEG_INDEX = convert_segmentations_to_index(PROCESSED_SEGEMENTATION_DATASET, real=USE_REAL_DATA)
PSEG_INDEX

In [ ]:
import librosa
from dataset import get_audio_sample_at_idx

# print(get_audio_sample_at_idx(720, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET))

In [ ]:
importlib.reload(dataset)
from dataset import DatasetMichigan
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

INFO,AUDIO_SAMPLE = get_audio_sample_at_idx(56, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET)
MEL = DatasetMichigan.preproccessData(AUDIO_SAMPLE, features={"mel_spectrogram"},pad_audio=True, pad_samples = librosa.time_to_samples(1, sr=16000), sampling_rate = 16000)[1]

# for i in range(len(PSEG_INDEX)):
#     print(i)
#     get_audio_sample_at_idx(i, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET)


# Plot the mel spectrogram
print(INFO)
plt.figure(figsize=(10, 4))
librosa.display.specshow(MEL, sr=16000, hop_length=321, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.show()

Audio(AUDIO_SAMPLE, rate=16000)

In [ ]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from model import *
from hparams import *

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import  ConfusionMatrixDisplay

class Metrics:
    def __init__(self):
        self.buffer = {}
        self.results_buffer = []

    def update(self, out, tgt, loss):
        with torch.no_grad():
            out = out.argmax(dim=1)
            out = torch.flatten(out)
            tgt = torch.flatten(tgt)

            acc = accuracy_score(tgt.cpu(), out.cpu())
            f1 = f1_score(tgt.cpu(), out.cpu(), average='macro')

            batch_metric = {
                'loss': loss.item(),
                'accuracy': acc,
                'f1': f1,
            }



            for k in batch_metric:
                if k in self.buffer:
                    self.buffer[k].append(batch_metric[k])
                else:
                    self.buffer[k] = [batch_metric[k]]

            self.results_buffer.append((out.cpu(), tgt.cpu()))

    def get_value(self):
        for k in self.buffer:
            self.buffer[k] = sum(self.buffer[k]) / len(self.buffer[k])
        ret = self.buffer
        ret2 = self.results_buffer
        self.buffer = {}
        self.results_buffer = []

        gt= torch.cat([x[1] for x in ret2]).tolist()
        pred= torch.cat([x[0] for x in ret2]).tolist()
        report= classification_report(gt, pred, output_dict=False, digits=4)

        return ret, report,gt,pred


def validateModelonPregeneratedSegments(model, device, data):
    
    loss_func = nn.CrossEntropyLoss()
    metric = Metrics()
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        pbar = tqdm(range(len(PSEG_INDEX)), desc='Validation', unit='sample')
        final_predictions = []

        for sample_idx, _ in enumerate(pbar):
            info, sample = get_audio_sample_at_idx(sample_idx, PSEG_INDEX, PROCESSED_SEGEMENTATION_DATASET, PROCESSED_SEGMENTAION_AUDIO_DATASET)

            # mel_spectrogram_normalised_log_scale_torch, yin_normalised_torch, pyin_normalised_torch, word, tone_class = batch
            mel = DatasetMichigan.preproccessData(sample, features={"mel_spectrogram"},pad_audio=True, pad_samples = librosa.time_to_samples(1.5, sr=16000), sampling_rate = 16000)[1]
            mel_spectrogram_normalised_log_scale_torch = torch.from_numpy(mel).float()

            tone_class = info['toneclass']
            tone_class -= 1 # 0-index
            tone_class = torch.tensor(tone_class)
            tone_class = tone_class.unsqueeze(0)

            x = mel_spectrogram_normalised_log_scale_torch.to(device)
            x = x[None, None, :, :]
            
            tgt = tone_class.to(device)
            out = model(x)
            loss = loss_func(out, tgt)
            metric.update(out, tgt, loss)
            pbar.set_postfix({'Loss': loss.item()})
    metric_test, report,gt,pred = metric.get_value()

    ConfusionMatrixDisplay.from_predictions(gt, pred, display_labels=["1","2","3","4"]).plot()
    return metric_test, report,gt,pred


model = ToneEval_Base(input_shape=(1, 128, 75))
# fit(model, args=Hparams.args, learning_params=learning_params)
model.load_state_dict(torch.load('results/config1_testMV1/best_model.pth'))
validateModelonPregeneratedSegments(model, "cuda" )
